In [1]:
import torch
import torchvision

# model = torchvision.models.alexnet(pretrained=False)
# model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 7)

model = torchvision.models.squeezenet1_1(pretrained = False)
model.classifier[1] = torch.nn.Conv2d(512, 7, kernel_size = 1)
model.num_classes = 7

model.load_state_dict(torch.load('signDetection_V2_squeeze_conv.pth'))
device = torch.device('cuda')
model = model.to(device)

In [4]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x[0:224, int(224*0): int(224*2/4)] =(0,0,0)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=10)
image = widgets.Image(format='jpeg', width=224, height=224)
fast_slider = widgets.FloatSlider(description='fast', min=0.0, max=1.0, orientation='vertical')
left_slider = widgets.FloatSlider(description='left', min=0.0, max=1.0, orientation='vertical')
right_slider = widgets.FloatSlider(description='right', min=0.0, max=1.0, orientation='vertical')
slow_slider = widgets.FloatSlider(description='slow', min=0.0, max=1.0, orientation='vertical')
stop_slider = widgets.FloatSlider(description='stop', min=0.0, max=1.0, orientation='vertical')
nothing_slider = widgets.FloatSlider(description='nothing', min=0.0, max=1.0, orientation='vertical')
attention_slider = widgets.FloatSlider(description='attention', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, fast_slider, left_slider, right_slider, slow_slider, stop_slider, nothing_slider, attention_slider]))

from jetbot import Robot
robot = Robot()

In [6]:
import torch.nn.functional as F
import time

def update(change):
    global fast_slider, left_slider, right_slider, slow_slider, stop_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_fast = float(y.flatten()[1])
    prob_left = float(y.flatten()[2])
    prob_right = float(y.flatten()[4])
    prob_slow = float(y.flatten()[5])
    prob_stop = float(y.flatten()[6])
    prob_nothing = float(y.flatten()[3])
    prob_attention = float(y.flatten()[0])
    # prob_worker = float(y.flatten()[5])
    
    fast_slider.value = prob_fast
    left_slider.value = prob_left
    right_slider.value = prob_right
    slow_slider.value = prob_slow
    stop_slider.value = prob_stop
    nothing_slider.value = prob_nothing
    attention_slider.value = prob_attention
    
    #if prob_blocked < 0.5:
    #    robot.forward(0.4)
    #else:
    #    robot.left(0.4)
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

In [7]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [8]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()